In [1]:
import numpy as np
import pandas as pd
import gdal
from osgeo import ogr
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, Sequential
import csv


In [2]:
path = '.'
scenes = ['05bc615a9b0e1159t', '72dba3e82f782f67t', '590dd08f71056cacv', '2899cfb18883251bt', 'b1844cde847a3942v', 'cbe4ad26fe73f118t', 'e98ca5aba8849b06t']

## Preprocessing CSVs


In [3]:
# Read in the training and validation labels
train_y_df = pd.read_csv(os.path.join(path, 'train.csv'), quoting=csv.QUOTE_NONE, error_bad_lines=False)
val_y_df = pd.read_csv(os.path.join(path, 'validation.csv'), quoting=csv.QUOTE_NONE, error_bad_lines=False)

In [4]:
train_y_df.head()

,detect_lat,detect_lon,vessel_length_m,source,detect_scene_row,detect_scene_column,is_vessel,is_fishing,distance_from_shore_km,scene_id,confidence,top,left,bottom,right,detect_id
0,5.662924,4.842429,NaN,ais,16722,22703,NaN,NaN,9999.990000,e42a50089e03990ft,LOW,NaN,NaN,NaN,NaN,e42a50089e03990ft_005.66292355123628965430_004...
1,5.830786,4.794394,NaN,ais,14867,22165,False,NaN,9999.990000,e42a50089e03990ft,HIGH,NaN,NaN,NaN,NaN,e42a50089e03990ft_005.83078557395605034941_004...
2,5.650289,5.076500,NaN,ais,16853,25297,False,NaN,4.120485,e42a50089e03990ft,HIGH,NaN,NaN,NaN,NaN,e42a50089e03990ft_005.65028885294942995188_005...
3,5.865495,4.938335,NaN,ais,14478,23758,False,NaN,4.344042,e42a50089e03990ft,HIGH,NaN,NaN,NaN,NaN,e42a50089e03990ft_005.86549528127105990194_004...
4,5.777973,4.910044,NaN,ais,15447,23448,False,NaN,9999.990000,e42a50089e03990ft,HIGH,NaN,NaN,NaN,NaN,e42a50089e03990ft_005.77797292764882008953_004...


In [5]:
val_y_df.head()

,detect_lat,detect_lon,vessel_length_m,source,detect_scene_row,detect_scene_column,is_vessel,is_fishing,distance_from_shore_km,scene_id,confidence,top,left,bottom,right,detect_id
0,44.287964,13.041733,NaN,manual,9923,23071,True,NaN,9999.990000,264ed833a13b7f2av,LOW,9918.0,23068.0,9928.0,23074.0,264ed833a13b7f2av_044.28796374000000213300_013...
1,44.617888,12.357101,NaN,manual,6105,17727,True,NaN,6.252201,264ed833a13b7f2av,LOW,6099.0,17725.0,6111.0,17729.0,264ed833a13b7f2av_044.61788760999999681189_012...
2,44.288475,13.710885,NaN,manual,10023,28410,True,NaN,9999.990000,264ed833a13b7f2av,MEDIUM,10018.0,28408.0,10028.0,28412.0,264ed833a13b7f2av_044.28847472999999723697_013...
3,44.066985,13.249822,NaN,ais/manual,12415,24679,False,NaN,9999.990000,264ed833a13b7f2av,HIGH,12412.0,24676.0,12418.0,24683.0,264ed833a13b7f2av_044.06698538999999925636_013...
4,44.067516,13.249182,NaN,ais/manual,12409,24674,False,NaN,9999.990000,264ed833a13b7f2av,HIGH,12406.0,24670.0,12412.0,24679.0,264ed833a13b7f2av_044.06751578000000080237_013...


In [4]:
# The dataframes include labels for all scenes in the dataset but we are working with a smaller subset as the full set is over 2 terabytes

# Extract labels for only our scenes
train_y_df = train_y_df[train_y_df['scene_id'].isin(scenes)]
val_y_df = val_y_df[val_y_df['scene_id'].isin(scenes)]

# Remove Nan from is_vessel
train_y_df = train_y_df[np.logical_or(train_y_df['is_vessel'] == True, train_y_df['is_vessel'] == False)]
val_y_df = val_y_df[np.logical_or(val_y_df['is_vessel'] == True, val_y_df['is_vessel'] == False)]

In [5]:
training_scenes = train_y_df.scene_id.unique()
validation_scenes = val_y_df.scene_id.unique()

print(training_scenes)
print(validation_scenes)

['2899cfb18883251bt' '72dba3e82f782f67t' 'e98ca5aba8849b06t'
 'cbe4ad26fe73f118t' '05bc615a9b0e1159t']
['590dd08f71056cacv' 'b1844cde847a3942v']


In [6]:
# Import VH scenes (features)
scene_590dd08f71056cacv = gdal.Open(os.path.join(path, '590dd08f71056cacv/VH_dB.tif'))
scene_b1844cde847a3942v = gdal.Open(os.path.join(path, 'b1844cde847a3942v/VH_dB.tif'))
scene_05bc615a9b0e1159t = gdal.Open(os.path.join(path, '05bc615a9b0e1159t/VH_dB.tif'))
scene_cbe4ad26fe73f118t = gdal.Open(os.path.join(path, 'cbe4ad26fe73f118t/VH_dB.tif'))
scene_e98ca5aba8849b06t = gdal.Open(os.path.join(path, 'e98ca5aba8849b06t/VH_dB.tif'))
scene_72dba3e82f782f67t = gdal.Open(os.path.join(path, '72dba3e82f782f67t/VH_dB.tif'))
scene_2899cfb18883251bt = gdal.Open(os.path.join(path, '2899cfb18883251bt/VH_dB.tif'))

train_scenes_list = [scene_2899cfb18883251bt, scene_72dba3e82f782f67t, scene_e98ca5aba8849b06t, scene_cbe4ad26fe73f118t, scene_05bc615a9b0e1159t]
val_scense_list = [scene_590dd08f71056cacv, scene_b1844cde847a3942v]

In [7]:
# Open the SAR data into array
train_scenes_arrays = [train_scenes_list[i].GetRasterBand(1).ReadAsArray() for i in range(0, len(train_scenes_list))]
val_scenes_arrays = [val_scense_list[i].GetRasterBand(1).ReadAsArray() for i in range(0, len(val_scense_list))]

In [11]:
# Set No Data == to np.nan instead of -32769.0
# for a in train_scenes_arrays:
#     a[a == -32768.0] = np.nan

# for a in val_scenes_arrays:
#     a[a == -32768.0] = np.nan

In [12]:
# Visualize scenes
# scene_id = ['2899cfb18883251bt', '72dba3e82f782f67t', 'e98ca5aba8849b06t', 'cbe4ad26fe73f118t', '05bc615a9b0e1159t']

# plt.figure(figsize=(20,20))
# for i in range(len(train_arr)):
#     plt.subplot(5,5,i+1)
#     plt.imshow(train_arr[i], interpolation=None, cmap='gray', vmin=np.nanmin(train_arr[i]), vmax=np.nanmax(train_arr[i]))
#     plt.xticks([])
#     plt.yticks([])
#     plt.grid(False)
#     plt.title(scene_id[i])

In [10]:
train_X = []
train_y = []
si = 0
i = 0

# Iterate through each scene in the 
for scene in train_scenes_arrays:

    for index, r in train_y_df[train_y_df['scene_id'] == training_scenes[si]].iterrows():
        
        row = r.detect_scene_row
        col = r.detect_scene_column
        label = 1 if r.is_vessel else 0 # change to boolean
        subset = scene[row-128:row+128,col-128:col+128] # all vessels are centered...probably bad

        if subset.min() != -32768.0:
            train_X.append(subset)
            train_y.append(label)
        i += 1
    
    si += 1

train_X = np.array(train_X)
train_y = np.array(train_y)

In [10]:
val_X = []
val_y = []
si = 0

for scene in val_scenes_arrays:

    for index, r in val_y_df[val_y_df['scene_id'] == validation_scenes[si]].iterrows():
        row = r.detect_scene_row
        col = r.detect_scene_column
        label = 1 if r.is_vessel else 0 # change to boolean
        subset = scene[row-128:row+128,col-128:col+128] # all vessels are centered...bad
        if subset.min() != -32768.0:
            val_X.append(subset)
            val_y.append(label)
        
        
    si += 1

val_X = np.array(val_X)
val_y = np.array(val_y)

In [11]:
# Normalize data
train_X = (train_X - train_X.min()) / (train_X.max() - train_X.min())

val_X = (val_X - val_X.min()) / (val_X.max() - val_X.min())

In [12]:
data_augmentation = Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.1),
  layers.RandomZoom(height_factor=(0,0.1)),
])

In [13]:
train_X_aug = data_augmentation(train_X)
val_X_aug = data_augmentation(val_X)

In [18]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(rate=0.2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

In [19]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='sigmoid',  kernel_regularizer='l2'))
model.add(layers.Dropout(rate=0.1))
model.add(layers.Dense(2))


In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 62, 62, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 60, 60, 64)       

In [41]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(val_X_aug, val_y, epochs=10, 
                    validation_split=0.2)


Epoch 1/10
34/34 [==============================] - 17s 496ms/step - loss: 0.6666 - accuracy: 0.6431 - val_loss: 0.5298 - val_accuracy: 0.8148
Epoch 2/10
34/34 [==============================] - 18s 526ms/step - loss: 0.6470 - accuracy: 0.6561 - val_loss: 0.5379 - val_accuracy: 0.8148
Epoch 3/10
34/34 [==============================] - 17s 486ms/step - loss: 0.6545 - accuracy: 0.6478 - val_loss: 0.5390 - val_accuracy: 0.8148
Epoch 4/10
34/34 [==============================] - 17s 493ms/step - loss: 0.6538 - accuracy: 0.6450 - val_loss: 0.5318 - val_accuracy: 0.8148
Epoch 5/10
34/34 [==============================] - 17s 512ms/step - loss: 0.6562 - accuracy: 0.6533 - val_loss: 0.5699 - val_accuracy: 0.8148
Epoch 6/10
34/34 [==============================] - 17s 504ms/step - loss: 0.6487 - accuracy: 0.6468 - val_loss: 0.5294 - val_accuracy: 0.8148
Epoch 7/10
34/34 [==============================] - 18s 537ms/step - loss: 0.6536 - accuracy: 0.6533 - val_loss: 0.5349 - val_accuracy: 0.8148

In [22]:
# Evaluate the model on the train data
train_loss, train_acc = model.evaluate(val_X,  val_y, verbose=2)
print(f'Train accuracy: {100*train_acc:.2f}%')

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(train_X,  train_y, verbose=2)
print(f'Test accuracy: {100*test_acc:.2f}%')

43/43 - 4s - loss: 0.6258 - accuracy: 0.6872 - 4s/epoch - 97ms/step
Train accuracy: 68.72%
13/13 - 1s - loss: 0.4846 - accuracy: 0.9173 - 1s/epoch - 99ms/step
Test accuracy: 91.73%


In [39]:
# Predict 
predictions = model.predict(val_X)

43/43 [==============================] - 4s 101ms/step


In [40]:
predictions = tf.nn.softmax(predictions).numpy()
predictions

array([[0.3505623 , 0.6494376 ],
       [0.35067677, 0.6493233 ],
       [0.35036626, 0.6496337 ],
       ...,
       [0.3514449 , 0.64855516],
       [0.34947953, 0.6505205 ],
       [0.35122627, 0.6487737 ]], dtype=float32)

In [25]:
class_list = [0,1]

In [26]:
predictions_class = np.argmax(predictions, axis=1)
print(predictions_class.shape)

(1346,)


In [27]:
class_num = 1 
print(class_list[class_num])
i1 = np.where((val_y == class_num) & (val_y == predictions_class)) # correct prediction
print('Indices of correct classification:', i1) 

i2 = np.where((val_y == class_num) & (val_y != predictions_class)) # incorrect prediction
print('Indices of incorrect classification:', i2) 

1
Indices of correct classification: (array([   0,    1,    2,    3,    4,    5,    6,    8,   11,   12,   13,
         14,   20,   22,   26,   27,   40,   43,   44,   48,   49,   50,
         51,   58,   59,   60,   61,   62,   63,   64,   65,   66,   67,
         68,   69,   75,   79,   81,   82,   83,   84,   87,   88,   89,
         90,   91,   93,   95,   96,   97,   99,  105,  106,  107,  109,
        110,  111,  112,  113,  114,  115,  116,  117,  118,  119,  120,
        123,  125,  126,  127,  128,  131,  132,  138,  139,  140,  141,
        142,  148,  150,  151,  152,  154,  155,  156,  160,  161,  163,
        164,  165,  166,  167,  168,  169,  172,  173,  174,  175,  176,
        177,  178,  179,  180,  181,  182,  184,  185,  186,  187,  190,
        191,  192,  193,  195,  196,  197,  200,  204,  206,  207,  208,
        209,  210,  214,  215,  217,  218,  219,  224,  227,  228,  231,
        232,  233,  234,  235,  236,  237,  238,  241,  242,  244,  245,
        246, 

In [43]:
class_list[class_num]

1

In [ ]:
# index = 20
# plt.figure(figsize = (10,10)) # set figure size 
# plt.imshow(val_X[index]) # take first element from list of indices  
# plt.title(f'Correct={class_list[class_num]}, predicted class={class_list[predictions_class[index]]}') # shows name of class from index
# plt.show()

In [22]:
# Plotting loss, accuracy
# plt.subplot(1,2,1)
# plt.plot(history.history['loss'], label='loss')
# plt.plot(history.history['val_loss'], label = 'val_loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# #plt.ylim([0.5, 1])
# plt.legend(loc='lower right')

# plt.subplot(1,2,2)
# plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.ylim([0, 1])
# plt.legend(loc='lower right')

# plt.tight_layout()
# plt.show()

: 

: 

In [14]:
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(input_shape=(256, 256, 3),
                                               include_top=False,
                                               weights='imagenet',
                                               include_preprocessing=False)

In [17]:
rgb_batch = np.repeat(val_X_aug[..., np.newaxis], 3, -1)
print(rgb_batch.shape)

(1346, 256, 256, 3)


In [18]:
# Project 1d image into 3 dimensions for use in EfficientNet which requires RGB
rgb_batch2 = np.repeat(train_X_aug[..., np.newaxis], 3, -1)
print(rgb_batch2.shape)

(411, 256, 256, 3)


In [19]:
features = base_model.predict(rgb_batch2)
print(features.shape)

13/13 [==============================] - 5s 232ms/step
(411, 8, 8, 1280)


In [20]:
base_model.trainable = False

In [21]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
features_average = global_average_layer(features)
print(features_average.shape)

(411, 1280)


In [22]:
# 2 classes
prediction_layer = tf.keras.layers.Dense(2)
prediction = prediction_layer(features_average)
print(prediction.shape)

(411, 2)


In [24]:
# data_augmentation = tf.keras.Sequential([
#   tf.keras.layers.RandomFlip("horizontal_and_vertical"),
#   tf.keras.layers.RandomRotation(0.1),
#   tf.keras.layers.RandomZoom(height_factor=(0,0.1)),
# ])

inputs = tf.keras.Input(shape=(256, 256, 3))
# x = data_augmentation(inputs)
x = base_model(inputs, training=False)
x = global_average_layer(x)
#x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model_pretrained = tf.keras.Model(inputs, outputs)

In [25]:
model_pretrained.compile(tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [26]:
model_pretrained.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 efficientnetv2-b0 (Function  (None, 8, 8, 1280)       5919312   
 al)                                                             
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 2)                 2562      
                                                                 
Total params: 5,921,874
Trainable params: 2,562
Non-trainable params: 5,919,312
_________________________________________________________________


In [27]:
history = model_pretrained.fit(rgb_batch, val_y, epochs=20, shuffle=True, use_multiprocessing=True, batch_size=16, validation_split=0.2)

Epoch 1/20
68/68 [==============================] - 17s 169ms/step - loss: 0.6712 - accuracy: 0.6310 - val_loss: 0.5675 - val_accuracy: 0.8148
Epoch 2/20
68/68 [==============================] - 10s 154ms/step - loss: 0.6467 - accuracy: 0.6543 - val_loss: 0.5197 - val_accuracy: 0.8148
Epoch 3/20
68/68 [==============================] - 10s 152ms/step - loss: 0.6385 - accuracy: 0.6599 - val_loss: 0.5684 - val_accuracy: 0.8111
Epoch 4/20
68/68 [==============================] - 11s 157ms/step - loss: 0.6352 - accuracy: 0.6552 - val_loss: 0.6023 - val_accuracy: 0.7852
Epoch 5/20
68/68 [==============================] - 10s 144ms/step - loss: 0.6337 - accuracy: 0.6468 - val_loss: 0.5714 - val_accuracy: 0.8111
Epoch 6/20
68/68 [==============================] - 10s 152ms/step - loss: 0.6423 - accuracy: 0.6571 - val_loss: 0.6087 - val_accuracy: 0.7593
Epoch 7/20
68/68 [==============================] - 10s 152ms/step - loss: 0.6320 - accuracy: 0.6506 - val_loss: 0.5539 - val_accuracy: 0.8148

In [28]:
train_loss, train_acc = model_pretrained.evaluate(rgb_batch,  val_y, verbose=2)
print(f'Train accuracy: {100*train_acc:.2f}%')

# Evaluate the model on the test data
test_loss, test_acc = model_pretrained.evaluate(rgb_batch2,  train_y, verbose=2)
print(f'Test accuracy: {100*test_acc:.2f}%')

43/43 - 11s - loss: 0.5960 - accuracy: 0.6984 - 11s/epoch - 246ms/step
Train accuracy: 69.84%
13/13 - 3s - loss: 0.4205 - accuracy: 0.9173 - 3s/epoch - 254ms/step
Test accuracy: 91.73%


In [30]:
predictions = model_pretrained.predict(rgb_batch2)

# Convert predictions to probabilities using softmax
# https://en.wikipedia.org/wiki/Softmax_function
predictions = tf.nn.softmax(predictions).numpy()

13/13 [==============================] - 5s 236ms/step


In [36]:
np.unique(val_y_df.is_vessel, return_counts=True)

(array([False, True], dtype=object), array([426, 926], dtype=int64))

In [37]:
np.unique(train_y_df.is_vessel, return_counts=True)

(array([False, True], dtype=object), array([ 34, 378], dtype=int64))